In [ ]:
# Testing OPENSPEIL LIB
import pyspiel
import open_spiel
import numpy as np

In [ ]:
import open_spiel.python
import open_spiel.python.algorithms
import open_spiel.python.algorithms.nfsp


open_spiel.python.algorithms.nfsp.NFSP(
    #DONE
)

In [ ]:
import pyspiel
import open_spiel
import numpy as np
game = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 4, "numRanks":13, "numHoleCards": 2, "numBoardCards": "0 3", "bettingAbstraction": "fcpa", "numRounds":2})
import copy
class WrapperEnv:
    def __init__(self,game):
        self.game= game
        self.state = game.new_initial_state()
        self.agent_selection = str(self.state.current_player())
        self.traverser = None
    
    def reset(self, seed=None):
        self.state = self.game.new_initial_state()
        while self.state.is_chance_node():
            self.state.apply_action(np.random.choice(self.state.legal_actions()))
        self.agent_selection =  str(self.state.current_player())
        return self.obs()
    
    def step(self, action):
        if self.state.is_chance_node():
            while self.state.is_chance_node():
                self.state.apply_action(np.random.choice(self.state.legal_actions()))

        else:
            self.state.apply_action(action)
            if self.state.is_chance_node():
                while self.state.is_chance_node():
                    self.state.apply_action(np.random.choice(self.state.legal_actions()))
        
        if self.state.is_terminal():
            return self.obs()
        else:
            # store = copy.deepcopy(self.state)
            while self.state.is_chance_node():
                self.state.apply_action(np.random.choice(self.state.legal_actions()))
            # print("3")
            # print(self.state.is_terminal())
            # if self.state.is_terminal():
            #     print("store:", store)
            # print(self.state)
            # print(self.state.legal_actions_mask(int(self.agent_selection)))
            # print("3")
            self.agent_selection =  str(self.state.current_player())

            return self.obs()
    
    def last(self):
        return self.obs()
    
    def obs(self):
        return {"observation":self.state.observation_tensor(int(self.agent_selection)), "action_mask":np.asarray(self.state.legal_actions_mask(int(self.agent_selection)))}, self.state.player_reward(self.traverser) if self.traverser is not None else self.state.player_return(int(self.agent_selection)), self.state.is_terminal(), False, "OPENSPIEL"

env = WrapperEnv(game)


In [ ]:
from pettingzoo.classic import texas_holdem_v4
import copy
from agent_configs.cfr_config import CFRConfig
from active_player import ActivePlayer
from cfr_agent import CFRAgent
import torch
from cfr_network import CFRNetwork

hidden_dim = 128
input_dim = 108
output_dim = 4
num_players = 2
replay_buffer_size = 4000000
minibatch_size = 10000
steps_per_epoch = 3000
traversals = 3000
training_steps = 200
lr = 0.0001
optimizer = None
p_v_networks = {'input_shape':input_dim, 'output_shape':output_dim, 'hidden_size':hidden_dim, 'learning_rate':lr, 'optimizer':optimizer}
active_player_obj = ActivePlayer(num_players)
config = CFRConfig(
    config_dict={'network': {'policy': p_v_networks, 'value': p_v_networks, 'num_players':num_players},
                 'replay_buffer_size':replay_buffer_size,
                 'minibatch_size':minibatch_size,
                 'steps_per_epoch':steps_per_epoch,
                 'traversals': traversals,
                 'training_steps': training_steps,
                 'active_player_obj': active_player_obj,
                 },
    game_config={'num_players':num_players,
                 'observation_space':108,
                 'action_space':4,},
)
sampling = ["MC", "Full"]
for i in sampling:
    model = CFRAgent(env=env,config=config, name="CFR_FHP" + i)
    model.train(sampling=i)

Iteration 141 done
Iteration 142 done
Iteration 143 done
Iteration 144 done
Iteration 145 done
Iteration 146 done
Iteration 147 done
Iteration 148 done
Iteration 149 done
Iteration 150 done
Iteration 151 done
Iteration 152 done
Iteration 153 done
Iteration 154 done
Iteration 155 done
Iteration 156 done
Iteration 157 done
Iteration 158 done
Iteration 159 done
Iteration 160 done
Checkpointing at 3799544 nodes touched
Checkpointing at 80.0% of training steps, i.e 160 iterations
reshaped_iternum torch.Size([10000, 1])
outputs-batch[2].detach() torch.Size([10000, 4])
((outputs - batch[2].detach()) ** 2) torch.Size([10000, 4])
reshaped_iternum torch.Size([10000, 1])
outputs-batch[2].detach() torch.Size([10000, 4])
((outputs - batch[2].detach()) ** 2) torch.Size([10000, 4])
reshaped_iternum torch.Size([10000, 1])
outputs-batch[2].detach() torch.Size([10000, 4])
((outputs - batch[2].detach()) ** 2) torch.Size([10000, 4])
reshaped_iternum torch.Size([10000, 1])
outputs-batch[2].detach() torch.S

In [ ]:
testgame = texas_holdem_v4.env()
testgame.reset()

In [ ]:
testgame.last()
testgame.agent_selection = 'player_1'

In [ ]:
testgame.step(2)

In [ ]:
print(testgame.last())